In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
import json
import string
import pymorphy2
import lightfm

from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm_notebook
from multiprocessing import cpu_count, Pool

In [2]:
punctuation = string.punctuation
mapping = str.maketrans(punctuation, ' ' * len(punctuation))
ma = pymorphy2.MorphAnalyzer()

def normalize_text(s):
    normalized_splitted = [ma.normal_forms(word)[0] for word in s.translate(mapping).lower().split()]
    return " ".join(normalized_splitted)

def normalize_line(line):
    item = json.loads(line)
    item['content'] = normalize_text(item['content'])
    item['title'] = normalize_text(item['title'])
    if isinstance(item['image'], float):
        item['image'] = np.zeros((96,))
    else:
        item['image'] = np.array(item['image'])
    return item

In [3]:
with open('items.json') as items_json:
    with Pool(cpu_count()) as pool:
        items_json_list = list(pool.imap(normalize_line, items_json))
        
items = pd.DataFrame(items_json_list)
items.set_index('itemId')
items.head()

,content,itemId,image,title
0,"Согласитесь, дорогие любители собак, до чего ж...",0,"[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",Пять забавных «морщинистых» пород собак
1,"Контуры Третьей Поперечной улицы, состоявшей ...",1,"[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",История улицы Ирининской в Гомеле
2,Источник: http://infodays.ru Вообще он как-то ...,2,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",Зачем Дудь всё время спрашивает гостей програм...
3,41-летняя Светлана Зейналова решила окрестить ...,3,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",Светлана Зейналова крестила младшую дочь
4,«Организованные преступные группировки ГБАО де...,4,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...","ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [4]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/valeriy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
items_ = pd.read_csv('../../data/processed/processed_items.csv', index_col='itemId')
items_['image'] = items['image']
del items
items = items_
items.head()

,content,image,title
itemId,,,
0,согласиться дорогой любитель собака до что же ...,"[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",пять забавный морщинистый порода собака
1,контур три поперечный улица состоять до недавн...,"[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",история улица ирининский в гомель
2,источник http infodays ru вообще он как то сам...,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",зачем дудь весь время спрашивать гость програм...
3,41 летний светлана зейналов решить окрестить 5...,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",светлана зейналов крестить младший дочь
4,организовать преступный группировка гбао делат...,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...",гкнб бандит в гбао делать вид что расстаться с...


In [9]:
items['title'].fillna('', inplace=True)
items['content'].fillna('', inplace=True)

In [10]:
vectorizer = TfidfVectorizer(lowercase=False,
                             ngram_range=(1, 2),
                             min_df=100,
                             max_df=0.01,
                             stop_words=stopwords.words('russian'))
title_tfidf = vectorizer.fit_transform(items['title'])
title_tfidf.shape

(328050, 3659)

In [11]:
vectorizer = TfidfVectorizer(lowercase=False,
                             min_df=100,
                             max_df=0.01,
                             stop_words=stopwords.words('russian'))
content_tfidf = vectorizer.fit_transform(items['content'])
content_tfidf.shape

(328050, 23589)

In [12]:
item_features = scipy.sparse.hstack((scipy.sparse.eye(items.shape[0]), 
                                     title_tfidf,
                                     content_tfidf,
                                     np.array(items['image'].to_list())),
                                    format='csr')

In [13]:
import gc
gc.collect()

55

In [14]:
ratings = []
row = []
col = []

train_lines = sum(1 for line in open('train.json','r'))

with open('train.json') as train_file:
    for i, line in enumerate(tqdm_notebook(train_file, total=train_lines)):
        json_line = json.loads(line)
        for item, rating in json_line['trainRatings'].items():
            ratings.append(int(rating))
            row.append(i)
            col.append(int(item))
train_int = scipy.sparse.coo_matrix((ratings, (row, col)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
del ratings, row, col

In [16]:
gc.collect()

3

In [ ]:
model = lightfm.LightFM(no_components=64, loss='warp', learning_schedule='adadelta', random_state=42)
model.fit(train_int, epochs=25, num_threads=cpu_count(), item_features=item_features, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17


In [ ]:
sample = pd.read_csv('random_benchmark.csv')
sample['y_pred'] = model.predict(
    sample['userId'].values,
    sample['itemId'].values,
    item_features=item_features,
    num_threads=cpu_count(),
)
sample.sort_values(['userId', 'y_pred'], ascending=[True, False], inplace=True)
sample.drop(columns=['y_pred'], inplace=True)
sample.to_csv('lightfm_everything_warp_25_epochs_64_comps.csv', index=False)

In [ ]:
!kaggle competitions submit -c recsys-iad-challenge -f lightfm_everything_warp_25_epochs_64_comps.csv -m "everything tfidf warp loss 25 epochs"

In [57]:
!kaggle competitions submit -c recsys-iad-challenge -f lightfm_everything_warp_5_epochs.csv -m "everything tfidf warp loss 5 epochs"

100%|██████████████████████████████████████| 35.7M/35.7M [00:07<00:00, 4.76MB/s]
Successfully submitted to Articles RecSys